# Criar uma pasta nomeada com a sigla da UF para cada estado encontrado no arquivo CSV

In [2]:
import os
import pandas as pd

# Função para criar pastas a partir de um arquivo CSV
def criar_pastas_por_estado(arquivo_csv, diretorio_base):
    # Lê o arquivo CSV
    df = pd.read_csv(arquivo_csv)
    
    # Verifica se a coluna 'Estado' existe no CSV
    if 'uf' not in df.columns:
        raise ValueError("O arquivo CSV deve conter uma coluna chamada 'uf'.")
    
    # Cria o diretório base, se não existir
    os.makedirs(diretorio_base, exist_ok=True)
    
    # Itera sobre os estados únicos encontrados no CSV
    for estado in df['uf'].unique():
        # Cria uma pasta para cada estado
        caminho_pasta = os.path.join(diretorio_base, estado)
        os.makedirs(caminho_pasta, exist_ok=True)
        print(f"Pasta criada: {caminho_pasta}")

# Parâmetros
arquivo_csv = 'exercicios/municipios-estados/csv/estados.csv'       # Caminho para o arquivo CSV
diretorio_base = 'exercicios/municipios-estados/processed_data' # Diretório onde as pastas serão criadas

# Execução do script
if __name__ == "__main__":
    criar_pastas_por_estado(arquivo_csv, diretorio_base)
    print("Pastas criadas com sucesso!")

Pasta criada: exercicios/municipios-estados/processed_data/RO
Pasta criada: exercicios/municipios-estados/processed_data/AC
Pasta criada: exercicios/municipios-estados/processed_data/AM
Pasta criada: exercicios/municipios-estados/processed_data/RR
Pasta criada: exercicios/municipios-estados/processed_data/PA
Pasta criada: exercicios/municipios-estados/processed_data/AP
Pasta criada: exercicios/municipios-estados/processed_data/TO
Pasta criada: exercicios/municipios-estados/processed_data/MA
Pasta criada: exercicios/municipios-estados/processed_data/PI
Pasta criada: exercicios/municipios-estados/processed_data/CE
Pasta criada: exercicios/municipios-estados/processed_data/RN
Pasta criada: exercicios/municipios-estados/processed_data/PB
Pasta criada: exercicios/municipios-estados/processed_data/PE
Pasta criada: exercicios/municipios-estados/processed_data/AL
Pasta criada: exercicios/municipios-estados/processed_data/SE
Pasta criada: exercicios/municipios-estados/processed_data/BA
Pasta cr

# Agrupar municipios por estado e por pasta

In [3]:
import pandas as pd
import os
import boto3

def criar_arquivos_por_estado(estados_csv, municipios_csv, diretorio_base):
    # Lê os arquivos CSV
    df_estados = pd.read_csv(estados_csv)
    df_municipios = pd.read_csv(municipios_csv)
    
    # Realiza o merge com base no campo 'codigo_uf'
    df_combinado = pd.merge(
        df_municipios,
        df_estados,
        how='left',  # Faz uma junção à esquerda
        on='codigo_uf'  # Coluna chave para junção
    )
    
    # Verifica se há municípios sem correspondência com um estado
    if df_combinado['uf'].isnull().any():
        print("Atenção: Alguns municípios não possuem correspondência com um estado!")
    
    # Cria o diretório base, se não existir
    os.makedirs(diretorio_base, exist_ok=True)
    
    # Itera por cada estado único no DataFrame
    for codigo_uf, grupo in df_combinado.groupby('codigo_uf'):
        # Obtém o nome do estado correspondente
        nome_estado = grupo['uf'].iloc[0]
        
        # Cria a pasta do estado, se não existir
        caminho_pasta = os.path.join(diretorio_base, nome_estado)
        os.makedirs(caminho_pasta, exist_ok=True)
        
        # Salva o arquivo CSV dentro da pasta do estado
        caminho_arquivo = os.path.join(caminho_pasta, 'cidades.csv')
        grupo.to_csv(caminho_arquivo, index=False)
        print(f"Arquivo gerado: {caminho_arquivo}")

# Parâmetros
estados_csv = 'exercicios/municipios-estados/csv/estados.csv'          # Caminho para o arquivo de estados
municipios_csv = 'exercicios/municipios-estados/csv/municipios.csv'    # Caminho para o arquivo de municípios
diretorio_base = 'exercicios/municipios-estados/processed_data/'    # Diretório base para salvar os arquivos

# Execução
if __name__ == "__main__":
    criar_arquivos_por_estado(estados_csv, municipios_csv, diretorio_base)
    print("Arquivos gerados com sucesso!")

Arquivo gerado: exercicios/municipios-estados/processed_data/RO/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/AC/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/AM/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/RR/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/PA/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/AP/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/TO/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/MA/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/PI/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/CE/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/RN/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/PB/cidades.csv
Arquivo gerado: exercicios/municipios-estados/processed_data/PE/cidades.csv
Arquivo gera

# Salva arquivos no Minio

In [4]:
import os
from dotenv import load_dotenv
import boto3
from botocore.exceptions import NoCredentialsError

load_dotenv()

def upload_para_minio(diretorio_base, bucket_name, minio_config):
    # Conecta ao MinIO
    s3_client = boto3.client(
        's3',
        endpoint_url=minio_config['endpoint_url'],
        aws_access_key_id=minio_config['access_key'],
        aws_secret_access_key=minio_config['secret_key']
    )
    
    # Percorre as pastas e arquivos no diretório base
    for estado in os.listdir(diretorio_base):
        caminho_pasta = os.path.join(diretorio_base, estado)
        if os.path.isdir(caminho_pasta):
            # Caminho do arquivo CSV
            caminho_arquivo = os.path.join(caminho_pasta, 'cidades.csv')
            
            if os.path.isfile(caminho_arquivo):
                # Define a chave do objeto no MinIO
                chave_no_bucket = f"exercicios/{estado}/cidades.csv"
                
                try:
                    # Faz upload do arquivo
                    s3_client.upload_file(caminho_arquivo, bucket_name, chave_no_bucket)
                    print(f"Upload concluído: {chave_no_bucket}")
                except NoCredentialsError:
                    print("Erro: Credenciais inválidas para o MinIO.")
                except Exception as e:
                    print(f"Erro ao fazer upload: {e}")

# Parâmetros
diretorio_base = 'exercicios/municipios-estados/processed_data/'  # Diretório onde estão os arquivos CSV
bucket_name = 'aula-06'        # Nome do bucket no MinIO

# Configurações do MinIO
minio_config = {
    'endpoint_url': os.environ.get('MINIO_ENDPOINT'),  # Substitua pelo seu endpoint
    'access_key': os.environ.get('MINIO_ACCESS_KEY'),               # Substitua pela sua access key
    'secret_key': os.environ.get('MINIO_SECRET_KEY')                # Substitua pela sua secret key
}

# Execução
if __name__ == "__main__":
    upload_para_minio(diretorio_base, bucket_name, minio_config)
    print("Upload para o MinIO concluído.")

Upload concluído: exercicios/AC/cidades.csv
Upload concluído: exercicios/AL/cidades.csv
Upload concluído: exercicios/AM/cidades.csv
Upload concluído: exercicios/AP/cidades.csv
Upload concluído: exercicios/BA/cidades.csv
Upload concluído: exercicios/CE/cidades.csv
Upload concluído: exercicios/DF/cidades.csv
Upload concluído: exercicios/ES/cidades.csv
Upload concluído: exercicios/GO/cidades.csv
Upload concluído: exercicios/MA/cidades.csv
Upload concluído: exercicios/MG/cidades.csv
Upload concluído: exercicios/MS/cidades.csv
Upload concluído: exercicios/MT/cidades.csv
Upload concluído: exercicios/PA/cidades.csv
Upload concluído: exercicios/PB/cidades.csv
Upload concluído: exercicios/PE/cidades.csv
Upload concluído: exercicios/PI/cidades.csv
Upload concluído: exercicios/PR/cidades.csv
Upload concluído: exercicios/RJ/cidades.csv
Upload concluído: exercicios/RN/cidades.csv
Upload concluído: exercicios/RO/cidades.csv
Upload concluído: exercicios/RR/cidades.csv
Upload concluído: exercicios/RS/

# Salvar arquivos no MongoDB

In [5]:
import os
from dotenv import load_dotenv
import pandas as pd
from pymongo import MongoClient

load_dotenv()

def salvar_dados_no_mongodb(diretorio_base, mongodb_config):
    # Conecta ao MongoDB
    client = MongoClient(mongodb_config['host'], mongodb_config['port'])
    db = client[mongodb_config['database']]
    
    # Coleção onde os dados serão salvos
    colecao = db[mongodb_config['collection']]
    
    # Limpa a coleção antes de inserir novos dados (opcional)
    colecao.delete_many({})
    print(f"Coleção '{mongodb_config['collection']}' limpa.")
    
    # Percorre as pastas e arquivos no diretório base
    for estado in os.listdir(diretorio_base):
        caminho_pasta = os.path.join(diretorio_base, estado)
        if os.path.isdir(caminho_pasta):
            # Caminho do arquivo CSV
            caminho_arquivo = os.path.join(caminho_pasta, 'cidades.csv')
            
            if os.path.isfile(caminho_arquivo):
                # Carrega os dados do CSV em um DataFrame
                df = pd.read_csv(caminho_arquivo)
                
                # Adiciona uma coluna com o nome do estado
                df['estado'] = estado
                
                # Converte o DataFrame para uma lista de dicionários e insere no MongoDB
                dados = df.to_dict(orient='records')
                colecao.insert_many(dados)
                print(f"Dados do estado '{estado}' inseridos no MongoDB.")

    print("Todos os dados foram inseridos no MongoDB com sucesso.")
    client.close()

# Parâmetros
diretorio_base = 'exercicios/municipios-estados/processed_data/'  # Diretório onde estão os arquivos CSV

# Configurações do MongoDB
mongodb_config = {
    'host': os.environ.get('MONGODB_ENDPOINT'),           # Endereço do MongoDB
    'port': 27017,                 # Porta do MongoDB
    'database': 'dados_estados',   # Nome do banco de dados
    'collection': 'municipios'     # Nome da coleção
}

# Execução
if __name__ == "__main__":
    salvar_dados_no_mongodb(diretorio_base, mongodb_config)
    print("Processo concluído.")


Coleção 'municipios' limpa.
Dados do estado 'AC' inseridos no MongoDB.
Dados do estado 'AL' inseridos no MongoDB.
Dados do estado 'AM' inseridos no MongoDB.
Dados do estado 'AP' inseridos no MongoDB.
Dados do estado 'BA' inseridos no MongoDB.
Dados do estado 'CE' inseridos no MongoDB.
Dados do estado 'DF' inseridos no MongoDB.
Dados do estado 'ES' inseridos no MongoDB.
Dados do estado 'GO' inseridos no MongoDB.
Dados do estado 'MA' inseridos no MongoDB.
Dados do estado 'MG' inseridos no MongoDB.
Dados do estado 'MS' inseridos no MongoDB.
Dados do estado 'MT' inseridos no MongoDB.
Dados do estado 'PA' inseridos no MongoDB.
Dados do estado 'PB' inseridos no MongoDB.
Dados do estado 'PE' inseridos no MongoDB.
Dados do estado 'PI' inseridos no MongoDB.
Dados do estado 'PR' inseridos no MongoDB.
Dados do estado 'RJ' inseridos no MongoDB.
Dados do estado 'RN' inseridos no MongoDB.
Dados do estado 'RO' inseridos no MongoDB.
Dados do estado 'RR' inseridos no MongoDB.
Dados do estado 'RS' inser